# Setup for invoking Anthropic and Titan Family of models via SAP GenAI Hub on SAP AI core

## Part 1: Use AI Core REST APIs

Use this notebook to invoke the AI Core REST APIs to send your payloads into LLMs hosted on SAP GenAI hub. The documentation for the APIs are provided [here](https://api.sap.com/api/AI_CORE_API/resource/Deployment).


## Part 2: Use SAP Generative AI Hub SDK
Sample code to invoke Anthropic and Titan Family of models using the ChatBedrock interface in the SAP Generative AI Hub SDK. Documentation [here](https://pypi.org/project/generative-ai-hub-sdk/). 



## Part 1:
### Load your AI Core credentials

After following the steps in the previous notebook to setup the AI Core instance, Configurations and Deployments for Bedrock models, you can invoke the models via the following approach. 

First, you should have your AI core creadentials in your ~/.aicore/config.json. If you do not, get it by creating a service key from [here](https://help.sap.com/docs/sap-ai-core/sap-ai-core-service-guide/create-service-key). We will read and consume the data from this location into temporary environment files for this notebook.

A sample config.json is below:

```sh
$ cat ~/.aicore/config.json  
{
  "AICORE_AUTH_URL": "<>.authentication.us10.hana.ondemand.com",
  "AICORE_CLIENT_ID": "sb-b...64",
  "AICORE_CLIENT_SECRET": "21...xc=",
  "AICORE_RESOURCE_GROUP": "default",
  "AICORE_BASE_URL": "https://api.ai.prod.<>.ondemand.com"
}
```

In [1]:
import os
import json
import base64

def load_config():
    with open(os.path.expanduser('~/.aicore/config.json')) as f:
        config = json.load(f)
    for key, value in config.items():
        os.environ[key] = value

In [2]:
# from https://help.sap.com/docs/sap-ai-core/sap-ai-core-service-guide/models-and-scenarios-in-generative-ai-hub or uncomment from below

# Multimodal models:
os.environ["TARGET_AI_CORE_MODEL"] = "anthropic--claude-3.5-sonnet"
# os.environ["TARGET_AI_CORE_MODEL"] = "anthropic--claude-3-opus"
# os.environ["TARGET_AI_CORE_MODEL"] = "anthropic--claude-3-sonnet"
# os.environ["TARGET_AI_CORE_MODEL"] = "anthropic--claude-3-haiku"

#Text based models:
# os.environ["TARGET_AI_CORE_MODEL"] = "amazon--titan-text-lite"
# os.environ["TARGET_AI_CORE_MODEL"] = "amazon--titan-text-express"

# Embedding models:
# os.environ["TARGET_AI_CORE_MODEL"] = "amazon--titan-embed-text"

# Uncomment the below line after pasting your deployment ID if you already know it
# os.environ["DEPLOYMENT_ID"] = "<>"

Creating a utility function to send API calls to AI Core instance repeatedly

In [4]:
import requests
import sys
import logging
import time
from util.logging import initLogger

TIME_RETRY_API_CALL = 20
TIMEOUT_API_CALL = 3600

log = logging.getLogger(__name__)
initLogger()


# Function to call a rest API
def call_api(
    type: str, url: str, headers: dict, data: dict = None, message: str = None
):
    timeNeeded = 0
    while timeNeeded < TIMEOUT_API_CALL:
        try:
            r = None
            # Send the request to retrieve the access token
            if type == "POST":
                r = requests.post(url=url, headers=headers, data=data)
            elif type == "GET":
                r = requests.get(url=url, headers=headers)
            # if the response is OK, return the JSON response
            if r.ok is True:
                log.success(f"{message}")
                return r.json()
            else:
                log.info(
                    f"response ({message}): {r.status_code} ({r.reason}): {r.text}"
                )
                log.warning(
                    f"Could not {message}! Re-trying in {TIME_RETRY_API_CALL} seconds..."
                )
                time.sleep(TIME_RETRY_API_CALL)
                timeNeeded += TIME_RETRY_API_CALL
        except requests.exceptions.RequestException as e:
            log.warning(str(e))
            log.error(f"Could not {message}! Exiting...")
            sys.exit(1)
    log.error(f"Could not {message} after {TIMEOUT_API_CALL} seconds! Exiting...")


In [5]:
from dataclasses import dataclass
from json import JSONEncoder


class AiCoreMetadataJsonEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__


@dataclass
class AiCoreMetadataDefinition:
    authUrl: str
    clientId: str
    clientSecret: str
    apiBase: str
    resourceGroup: str
    targetAiCoreModel: str
    apiAccessToken: str
    deploymentUrl : str

    def __getitem__(self, item):
        return getattr(self, item)

In [5]:
@dataclass
class AiCoreMetadata(AiCoreMetadataDefinition):
    def __init__(self):

        load_config()

        self.authUrl = os.environ.get("AICORE_AUTH_URL")
        self.clientId = os.environ.get("AICORE_CLIENT_ID")
        self.clientSecret = os.environ.get("AICORE_CLIENT_SECRET")
        self.resourceGroup = os.environ.get("AICORE_RESOURCE_GROUP")
        self.apiBase = os.environ.get("AICORE_BASE_URL")
        self.targetAiCoreModel = os.environ.get("TARGET_AI_CORE_MODEL")
        self.apiAccessToken = get_api_access_token(self)
        self.deploymentUrl = get_deployment_details_for_model(self)

In [6]:
def get_api_access_token(aiCoreMetadata: AiCoreMetadataDefinition) -> str:
    clientId = aiCoreMetadata.clientId
    clientSecret = aiCoreMetadata.clientSecret
    authUrl = aiCoreMetadata.authUrl

    # Create the authorization string
    authorizationString = f"{clientId}:{clientSecret}"
    # Encode the authorization string
    byte_data = authorizationString.encode("utf-8")
    # Base64 encode the byte data
    clientSecretBase64 = base64.b64encode(byte_data).decode("utf-8")

    # Create the URL to retrieve the access token
    aiCoreLocation = f"{authUrl}/oauth/token?grant_type=client_credentials"
    # Create the headers for the request
    headers = {"Authorization": f"Basic {clientSecretBase64}"}

    response = call_api(
        "POST",
        aiCoreLocation,
        headers,
        None,
        "retrieve access token from AI Core system",
    )
    # json_response = json.dumps(response, indent=2)

    return response["access_token"]

In [7]:
# Retrieve the deployment URL from the AI Core system metadata
def get_deployment_details_for_model(aiCoreMetadata: AiCoreMetadataDefinition):
    apiBase = aiCoreMetadata.apiBase
    token = aiCoreMetadata.apiAccessToken
    resourceGroup = aiCoreMetadata.resourceGroup

    # Create the URL to get the deployment 
    aiCoreLocation = f"{apiBase}/v2/lm/deployments"
    # Create the headers for the request
    headers = {}
    headers["AI-Resource-Group"] = resourceGroup
    headers["Authorization"] = f"Bearer {token}"
    allDeploymentDetails = None

    timeNeeded = 0
    message = f"retrieve deployment details for model id {aiCoreMetadata.targetAiCoreModel}"
    while timeNeeded < TIMEOUT_API_CALL:
        # Send the request to get the list of deployments
        response = call_api("GET", aiCoreLocation, headers, None, message)
        json_response = json.dumps(response, indent=2)
        log.check(
                f"API response from retrieveing deployment details for model id {aiCoreMetadata.targetAiCoreModel}:\n{json_response}"
        )

        allDeploymentDetails = response
    
        for resource in allDeploymentDetails["resources"]:
            model_name = resource["details"]["resources"]["backend_details"]["model"]["name"]
            if model_name == aiCoreMetadata.targetAiCoreModel:
                return resource["deploymentUrl"]
        
        log.warning(
                f"Could not {message} Re-trying in {TIME_RETRY_API_CALL} seconds..."
            )

        time.sleep(TIME_RETRY_API_CALL)
        timeNeeded += TIME_RETRY_API_CALL

    log.error(
        f"Could not retrieve deployment details for id '{aiCoreMetadata.targetAiCoreModel}'! Exiting..."
    )
    sys.exit(1)

In [8]:
# Retrieve the deployment URL from the AI Core system metadata
def get_deployment_details(aiCoreMetadata: AiCoreMetadataDefinition, deploymenId: str):
    apiBase = aiCoreMetadata.apiBase
    token = aiCoreMetadata.apiAccessToken
    resourceGroup = aiCoreMetadata.resourceGroup

    # Create the URL to create the configuration
    aiCoreLocation = f"{apiBase}/v2/lm/deployments/{deploymenId}"
    # Create the headers for the request
    headers = {}
    headers["AI-Resource-Group"] = resourceGroup
    headers["Authorization"] = f"Bearer {token}"
    deploymentDetails = None

    timeNeeded = 0
    message = f"retrieve deployment details for deployment id {deploymenId}"
    while timeNeeded < TIMEOUT_API_CALL:
        # Send the request to create the deployment
        response = call_api("GET", aiCoreLocation, headers, None, message)
        # json_response = json.dumps(response, indent=2)

        deploymentDetails = response
        deploymentUrl = deploymentDetails["deploymentUrl"]
        if deploymentUrl != "":
            log.success(f"AI Core deployment id '{deploymenId}' is now accessible!")
            return deploymentDetails
        else:
            log.warning(
                f"Could not {message} (deployment not finished)! Re-trying in {TIME_RETRY_API_CALL} seconds..."
            )

            time.sleep(TIME_RETRY_API_CALL)
            timeNeeded += TIME_RETRY_API_CALL

    log.error(
        f"Could not retrieve deployment details for id '{deploymenId}'! Exiting..."
    )
    sys.exit(1)

In [ ]:
# Extract the metadata for the AI Core system
ai_core_metadata = AiCoreMetadata()

In [10]:
# In case you have provided deployment ID directly instead of model id 
if ai_core_metadata.deploymentUrl == None:
    deploymentDetails = get_deployment_details(ai_core_metadata, os.environ["DEPLOYMENT_ID"])
    ai_core_metadata.deploymentUrl = deploymentDetails["deploymentUrl"]

In [11]:
# Retrieve the available AI models from the AI Core system
def invoke(aiCoreMetadata: AiCoreMetadataDefinition, 
           messages,
           max_tokens = 1000) -> str:

    token = aiCoreMetadata.apiAccessToken
    deploymentUrl = aiCoreMetadata.deploymentUrl
    
    # Create the URL to retrieve the available AI models
    aiCoreLocation = f"{deploymentUrl}/invoke"
    # Create the headers for the request
    headers = {
        "AI-Resource-Group": aiCoreMetadata.resourceGroup,
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}",
    }

    data = {}
    data["anthropic_version"] = "bedrock-2023-05-31"
    data["max_tokens"] = max_tokens
    data["messages"] = messages
    
    response = call_api(
        "POST",
        aiCoreLocation,
        headers,
        json.dumps(data),
        "sending invoke",
    )
    json_response = json.dumps(response, indent=2)

    return response

In [12]:
messages = [
        {
            "role": "user",
            "content": "Hello, Claude"
        }
    ]

In [ ]:
invoke(aiCoreMetadata= ai_core_metadata, 
       messages=messages)


## Part 2: Use SAP Generative AI Hub SDK
Sample code to invoke Anthropic and Titan Family of models using the ChatBedrock interface in the SAP Generative AI Hub SDK. Documentation [here](https://pypi.org/project/generative-ai-hub-sdk/). 

In [ ]:
! pip -q install generative-ai-hub-sdk==1.11.1 langchain_aws

In [ ]:
from gen_ai_hub.proxy.langchain.amazon import ChatBedrock
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

In [ ]:
proxy_client = get_proxy_client("gen-ai-hub")
llm_via_bedrock_on_GenAIHub = ChatBedrock(
        model_name="anthropic--claude-3-sonnet",
        proxy_client=proxy_client,
        temperature=0,
    )

In [ ]:
messages = [
        ( "system", "You are a helpful assistant" ),
        ( "user", "Hello, Claude" )
        ]

response_body = llm_via_bedrock_on_GenAIHub.invoke(messages)
print(response_body.content)